# Hybrid Zero Shot Learning Approach

In [5]:
import pandas as pd

<h2>Data Retrieval</h2>

In [41]:
# Loading in promise dataset
df = pd.read_csv("datasets/promise.csv")

df.head()

,INPUT,TYPE
0,The system shall refresh the display every 60 seconds.,PE
1,The application shall match the color of the schema set forth by Department of Homeland Security,LF
2,If projected the data must be readable. On a 10x10 projection screen 90% of viewers must be a...,US
3,The product shall be available during normal business hours. As long as the user has access to t...,A
4,If projected the data must be understandable. On a 10x10 projection screen 90% of viewers must...,US


<h2>Data Preprocessing</h2>

<h3>Stopword Removal</h3>

In [43]:
import nltk
from nltk.corpus import stopwords

# Define stop words
stop_words = set(stopwords.words('english'))

# Function to remove stopwords
def remove_stopwords(text):
    words = text.split()  # Split the text into words
    filtered_text = ' '.join([word for word in words if word.lower() not in stop_words])
    return filtered_text

# Apply stopword removal to the DataFrame column
df['INPUT'] = df['INPUT'].apply(remove_stopwords)

# Display the DataFrame
print(df)

    INPUT                                                                                                TYPE
0                                                         system shall refresh display every 60 seconds.  PE 
1                            application shall match color schema set forth Department Homeland Security  LF 
2    projected data must readable. 10x10 projection screen 90% viewers must able read Event / Activit...  US 
3    product shall available normal business hours. long user access client PC system available 99% t...   A 
4    projected data must understandable. 10x10 projection screen 90% viewers must able determine Even...  US 
5    product shall ensure accessed authorized users. product able distinguish authorized unauthorized...  SE 
6    product shall intuitive self-explanatory. : 90% new users shall able start display Events Activi...  US 
7                                               product shall respond fast keep up-to-date data display.  PE 
8         